## CIFAR 10

In [93]:
import fastai.models as mod

In [25]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [26]:
from fastai.conv_learner import *
PATH = "../fp16/data/cifar10/"
os.makedirs(PATH,exist_ok=True)

In [27]:
from fp16util import *

### Load Data

In [28]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
stats = (np.array([ 0.4914 ,  0.48216,  0.44653]), np.array([ 0.24703,  0.24349,  0.26159]))

In [29]:
def get_data(sz,bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='test', tfms=tfms, bs=bs)

In [30]:
bs=128

In [31]:

def BN_convert_float(module):
    '''
    BatchNorm layers to have parameters in single precision.
    Find all layers and convert them back to float. This can't
    be done with built in .apply as that function will apply
    fn to all modules, parameters, and buffers. Thus we wouldn't
    be able to guard the float conversion based on the module type.
    '''
    if isinstance(module, torch.nn.modules.batchnorm._BatchNorm):
        module.float()
    for child in module.children():
        BN_convert_float(child)
    return module


def network_to_half(network):
    return nn.Sequential(tofp16(), BN_convert_float(network.half()))

class tofp16(nn.Module):
    def __init__(self):
        super(tofp16, self).__init__()

    def forward(self, input):
        return input.half()

## Initial model

In [ ]:
%pdb off

In [43]:
from fastai.fp16 import *

In [44]:
from fastai.models.cifar10.resnext import resnext29_8_64

m = resnext29_8_64()
# m = network_to_half(m.cuda())
m = FP16(m.cuda())
# m = m.cuda()
bm = BasicModel(m, name='cifar10_rn29_8_64')
data = get_data(32,bs)

In [45]:
learn = ConvLearner(data, bm)
learn.unfreeze()
lr=1e-2; wd=5e-4

In [14]:
# learn.half()

In [ ]:
learn.load('fp32_model')

In [46]:
opt = torch.optim.SGD(m.parameters(), lr=.05)

In [47]:
crit = nn.CrossEntropyLoss().cuda()

In [48]:
fit(m, data, 1, opt, crit)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  0%|          | 0/391 [00:00<?, ?it/s]         


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Time: 0.04858803749084473 0.0015673637390136719 0.2192227840423584 0.04903531074523926
  2%|▏         | 8/391 [00:06<05:13,  1.22it/s]


KeyboardInterrupt: 

In [37]:
fit(m, data, 1, opt, crit)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  5%|▍         | 18/391 [00:08<02:53,  2.15it/s]Time: 0.048459529876708984 0.09595036506652832 0.022384166717529297



KeyboardInterrupt: 

In [ ]:
%time learn.fit(lr, 1, cycle_len=.25)

In [ ]:
# %time learn.fit(lr, 1, cycle_len=.25, loss_scale=128, fp16=True)

In [ ]:
%time learn.fit(lr, 1, cycle_len=.25, loss_scale=128, fp16=False)

In [ ]:
learn.model

In [ ]:
learn.save('fp16_model')

## Testing 32

In [18]:
from fastai.models.cifar10.resnext import resnext29_8_64

m = resnext29_8_64()
bm = BasicModel(m.cuda(), name='cifar10_rn29_8_64')
data = get_data(32,bs)

In [19]:
learn = ConvLearner(data, bm)
learn.unfreeze()
lr=1e-2; wd=5e-4

In [ ]:
learn.load('fp16_model')

In [ ]:
%time learn.fit(lr, 1, cycle_len=.25)

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  5%|▌         | 5/97 [00:02<00:51,  1.80it/s, loss=3.5]  

In [ ]:
learn.save('fp32_model')